<a href="https://colab.research.google.com/github/neethu2602/DSA_Class_Activities-/blob/main/ETL_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import sqlite3
url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"
df = pd.read_csv(url)

In [3]:
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2025-09-21T07:34:48.073Z,-13.616900,122.376400,10.00,4.90,mb,33.0,53.0,4.26100,0.72,...,2025-09-21T07:51:54.040Z,northwest of Australia,earthquake,7.86,1.890,0.112,25.0,reviewed,us,us
1,2025-09-21T07:29:37.335Z,61.388600,-146.597800,27.20,1.40,ml,NaN,NaN,NaN,1.04,...,2025-09-21T07:31:55.178Z,"31 km NNW of Valdez, Alaska",earthquake,NaN,0.300,NaN,NaN,automatic,ak,ak
2,2025-09-21T07:14:56.510Z,33.339833,-116.287833,11.74,0.97,ml,56.0,36.0,0.02893,0.21,...,2025-09-21T07:18:21.644Z,"12 km NE of Borrego Springs, CA",earthquake,0.20,0.300,0.148,27.0,automatic,ci,ci
3,2025-09-21T07:14:21.263Z,53.168000,161.211900,10.00,4.60,mb,41.0,113.0,1.55200,0.86,...,2025-09-21T07:35:16.040Z,"173 km E of Petropavlovsk-Kamchatsky, Russia",earthquake,6.66,1.835,0.073,56.0,reviewed,us,us
4,2025-09-21T07:01:58.208Z,67.361700,-156.744900,1.40,2.50,ml,NaN,NaN,NaN,0.51,...,2025-09-21T07:06:42.656Z,"51 km N of Kobuk, Alaska",earthquake,NaN,0.600,NaN,NaN,automatic,ak,ak


In [2]:
num_rec = len(df)
unique_places = df['place'].nunique()
print("Number of records: ", num_rec)
print("Unique locations: ", unique_places)

Number of records:  9928
Unique locations:  5263


In [9]:
df['time'] = pd.to_datetime(df['time'])
print(df.dtypes['time'])

datetime64[ns, UTC]


In [10]:
df = df.dropna(subset=['latitude', 'longitude', 'mag'])

In [11]:
df = df[df['mag'] >= 4.0]

In [12]:
df['day_of_week'] = df['time'].dt.day_name()

In [13]:
def classify_severity(mag):
    if mag < 4.0:
        return "Low"
    elif mag < 6.0:
        return "Moderate"
    else:
        return "High"

df['severity_level'] = df['mag'].apply(classify_severity)

In [19]:
print(df['place'].value_counts())

place
South Sandwich Islands region                   13
Drake Passage                                   10
south of the Fiji Islands                        8
Izu Islands, Japan region                        6
Fiji region                                      6
                                                ..
85 km E of Severo-Kuril’sk, Russia               1
182 km SE of Vilyuchinsk, Russia                 1
186 km E of Petropavlovsk-Kamchatsky, Russia     1
170 km SE of Vilyuchinsk, Russia                 1
42 km ESE of Severo-Kuril’sk, Russia             1
Name: count, Length: 733, dtype: int64


In [20]:
avg_max = df.groupby(df['time'].dt.date).agg(
    avg_magnitude=('mag', 'mean'),
    max_depth=('depth', 'max')
).reset_index()

In [22]:
avg_max.head()

,time,avg_magnitude,max_depth
0,2025-08-22,4.507692,515.056
1,2025-08-23,4.582979,565.087
2,2025-08-24,4.515217,499.800
3,2025-08-25,4.677778,126.766
4,2025-08-26,4.678571,612.812


In [23]:
df.to_csv("cleaned_earthquakes.csv", index=False)

In [24]:
df.to_csv("cleaned_earthquakes.csv", index=False)
avg_max.to_csv("earthquake_summary.csv", index=False)

#Flowchart
## **Load CSV data -->  (Convert time → datetime,Drop NaN (latitude, longitude, magnitude)) --> Filter mag ≥ 4.0, Add day_of_week, Classify severity_level --> Count quakes per place, Avg mag & max depth/day --> Save CSV --> store into SQLite table**

# **Extract**
Pulled earthquake data

# **Transform**

Cleaned missing values.

Converted time to datetime.

Filtered out small earthquakes (magnitude < 4.0).

Added day_of_week and severity_level classifications.

Generated summaries: per place and per day.

# **Load**
Saved clean data to CSV files and SQLite database

# **Insight from data**
Most earthquakes are moderate (4.0–6.0) and occur near tectonic plate boundaries, while strong quakes (6.0+) are rare and usually deeper underground.